In [1]:
import pandas as pd
from sklearn.metrics import roc_auc_score
import numpy as np
import gc
np.random.seed(7)

In [2]:
def bootstrap_auc(y_true, y_pred, n=1000):
    """
    Generate `n` bootstrap samples, evaluating `func`
    at each resampling. `bootstrap` returns a function,
    which can be called to obtain confidence intervals
    of interest.
    """
    simulations = list()
    sample_size = len(y_pred)
    num_tried = 0
    while len(simulations) < n and num_tried < 10000:
        num_tried += 1
        iteridx = np.random.choice(range(len(y_pred)), size=sample_size, replace=True)
        iterypred = [np.array(y_pred)[idx] for idx in iteridx]
        iterytrue = [np.array(y_true)[idx] for idx in iteridx]
        if len(set(iterytrue)) < 2:
            continue
        simulations.append(roc_auc_score(iterytrue, iterypred))
    simulations.sort()
    def ci(p):
        """
        Return 2-sided symmetric confidence interval specified
        by p.
        """
        u_pval = (1+p)/2.
        l_pval = (1-u_pval)
        l_indx = int(np.floor(n*l_pval))
        u_indx = int(np.floor(n*u_pval))
        return(np.mean(simulations), np.abs(simulations[l_indx]-simulations[u_indx])/2)
    result = ci(.95)
    return'{:.3f} ± {:.3f}'.format(result[0], result[1])

## Process all dataframes

In [3]:
cxr_dfs = []
for i in [1,2,3,4,5]:
    cxr_dfs.append(pd.read_csv('model_files/cxr_run_{}/cxr_test_result.csv'.format(i)))
cxr_df = pd.concat(cxr_dfs)[['Path', 'Pneumothorax']]
cxr_df.columns = ['Path', 'Pneumothorax_pred']
cxr_df = cxr_df.groupby(cxr_df['Path']).mean()
cxr_test_csv = pd.read_csv('test_dfs/cxr_test_df.csv')
cxr_df = cxr_df.merge(cxr_test_csv, on=['Path'])

nih_dfs = []
for i in [1,2,3,4,5]:
    nih_dfs.append(pd.read_csv('model_files/nih_run_{}/nih_test_result.csv'.format(i)))
nih_df = pd.concat(nih_dfs)[['Path', 'Pneumothorax']]
nih_df.columns = ['Path', 'Pneumothorax_pred']
nih_df = nih_df.groupby(nih_df['Path']).mean()
nih_test_csv = pd.read_csv('test_dfs/nih_test_df.csv')
nih_df = nih_df.merge(nih_test_csv, on=['Path'])
nih_df['study_id'] = nih_df.apply(lambda row: str(row['Patient ID'])+'/'+str(row['Follow-up #']), axis=1)

cxp_dfs = []
for i in [1,2,3,4,5]:
    cxp_dfs.append(pd.read_csv('model_files/cxp_run_{}/cxp_test_result.csv'.format(i)))
cxp_df = pd.concat(cxp_dfs)[['Path', 'Pneumothorax']]
cxp_df.columns = ['Path', 'Pneumothorax_pred']
cxp_df = cxp_df.groupby(cxp_df['Path']).mean()
cxp_test_csv = pd.read_csv('test_dfs/cxp_test_df.csv')
cxp_df = cxp_df.merge(cxp_test_csv, on=['Path'])
cxp_df['study_id'] = cxp_df.apply(lambda row: row['Path'].split('chexpert_full/train/')[1].split('/view')[0], axis=1)


## CXP

In [4]:
for val in cxp_df['Sex'].unique():
    gt_df = cxp_df[cxp_df['Sex'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))

for val in [20, 40, 60, 80, 100]:
    gt_df = cxp_df[(val-20 <= cxp_df['Age']) & (cxp_df['Age'] < val)]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))


Male: 0.907 ± 0.010
Female: 0.897 ± 0.015
20: 0.815 ± 0.126
40: 0.892 ± 0.021
60: 0.904 ± 0.015
80: 0.907 ± 0.014
100: 0.886 ± 0.036


## NIH

In [5]:
for val in nih_df['Patient Gender'].unique():
    gt_df = nih_df[nih_df['Patient Gender'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))

for val in [20, 40, 60, 80, 100]:
    gt_df = nih_df[(val-20 <= nih_df['Patient Age']) & (nih_df['Patient Age'] < val)]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))


F: 0.880 ± 0.018
M: 0.878 ± 0.026
20: 0.919 ± 0.043
40: 0.891 ± 0.023
60: 0.870 ± 0.023
80: 0.885 ± 0.040


# CXR

In [6]:
demo_df = pd.read_csv('test_dfs/cxr_demo_df.csv')
cxr_combo_df = cxr_df.merge(demo_df, on='subject_id', how='inner')
cxr_combo_df['ethnicity'] = cxr_combo_df['ethnicity'].apply(lambda x: 'OTHER' if x in ['AMERICAN INDIAN/ALASKA NATIVE', 'UNKNOWN', 'UNABLE TO OBTAIN'] else x)
cxr_combo_df['insurance'] = cxr_combo_df['insurance'].apply(lambda x: 'Other' if x == 'Medicare' else x)
cxr_df = cxr_combo_df[cxr_combo_df['anchor_age'] != 0]


In [8]:
for val in cxr_combo_df['gender'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['gender'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in [20, 40, 60, 80, 100]:
    gt_df = cxr_combo_df[(val-20 <= cxr_combo_df['anchor_age']) & (cxr_combo_df['anchor_age'] < val)]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in cxr_combo_df['ethnicity'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['ethnicity'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in cxr_combo_df['insurance'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['insurance'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))


F: 0.889 ± 0.015 (N=10501)
M: 0.888 ± 0.013 (N=11387)
20: 0.995 ± 0.013 (N=79)
40: 0.877 ± 0.027 (N=2398)
60: 0.894 ± 0.015 (N=7103)
80: 0.889 ± 0.015 (N=8949)
100: 0.887 ± 0.027 (N=3359)
BLACK/AFRICAN AMERICAN: 0.907 ± 0.023 (N=3832)
WHITE: 0.883 ± 0.011 (N=14649)
HISPANIC/LATINO: 0.883 ± 0.058 (N=1547)
OTHER: 0.872 ± 0.033 (N=3050)
ASIAN: 0.892 ± 0.033 (N=692)
Other: 0.888 ± 0.010
Medicaid: 0.902 ± 0.028


## NIH on CXR

In [10]:
# NIH ON CXR
nih_dfs = []
for i in [2,3,4,5]:
    nih_dfs.append(pd.read_csv('model_files/nih_run_{}/cxr_test_result.csv'.format(i)))
nih_df = pd.concat(nih_dfs)[['Path', 'Pneumothorax']]
nih_df.columns = ['Path', 'Pneumothorax_pred']
nih_df = nih_df.groupby(nih_df['Path']).mean()
nih_df = nih_df.merge(cxr_test_csv, on=['Path'])
#nih_df = nih_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]

demo_df = pd.read_csv('test_dfs/cxr_demo_df.csv')
cxr_combo_df = nih_df.merge(demo_df, on='subject_id', how='inner')
cxr_combo_df['ethnicity'] = cxr_combo_df['ethnicity'].apply(lambda x: 'OTHER' if x in ['UNKNOWN', 'UNABLE TO OBTAIN'] else x)
cxr_combo_df['insurance'] = cxr_combo_df['insurance'].apply(lambda x: 'Other' if x == 'Medicare' else x)
cxr_df = cxr_combo_df[cxr_combo_df['anchor_age'] != 0]

for val in cxr_combo_df['gender'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['gender'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in [20, 40, 60, 80, 100]:
    gt_df = cxr_combo_df[(val-20 <= cxr_combo_df['anchor_age']) & (cxr_combo_df['anchor_age'] < val)]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in cxr_combo_df['ethnicity'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['ethnicity'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in cxr_combo_df['insurance'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['insurance'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))


F: 0.753 ± 0.025 (N=10501)
M: 0.771 ± 0.020 (N=11387)
20: 0.771 ± 0.325 (N=79)
40: 0.821 ± 0.038 (N=2398)
60: 0.791 ± 0.025 (N=7103)
80: 0.748 ± 0.028 (N=8949)
100: 0.697 ± 0.050 (N=3359)
BLACK/AFRICAN AMERICAN: 0.650 ± 0.066 (N=3832)
WHITE: 0.759 ± 0.020 (N=14649)
HISPANIC/LATINO: 0.755 ± 0.091 (N=1547)
OTHER: 0.800 ± 0.036 (N=2976)
ASIAN: 0.834 ± 0.054 (N=692)
AMERICAN INDIAN/ALASKA NATIVE: 0.641 ± 0.168 (N=74)
Other: 0.763 ± 0.016 (N=20909)
Medicaid: 0.812 ± 0.046 (N=2874)


## CXP on CXR

In [12]:
#CXP ON CXR
cxp_dfs = []
for i in [1,2,3,4,5]:
    cxp_dfs.append(pd.read_csv('model_files/cxp_run_{}/cxr_test_result.csv'.format(i)))
cxp_df = pd.concat(cxp_dfs)[['Path', 'Pneumothorax']]
cxp_df.columns = ['Path', 'Pneumothorax_pred']
cxp_df = cxp_df.groupby(cxp_df['Path']).mean()
cxp_df = cxp_df.merge(cxr_test_csv, on=['Path'])
#cxp_df = cxp_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]

cxr_combo_df = cxp_df.merge(demo_df, on='subject_id', how='inner')
cxr_combo_df['ethnicity'] = cxr_combo_df['ethnicity'].apply(lambda x: 'OTHER' if x in ['UNKNOWN', 'UNABLE TO OBTAIN'] else x)
cxr_combo_df['insurance'] = cxr_combo_df['insurance'].apply(lambda x: 'Other' if x == 'Medicare' else x)
cxr_df = cxr_combo_df[cxr_combo_df['anchor_age'] != 0]

for val in cxr_combo_df['gender'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['gender'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in [20, 40, 60, 80, 100]:
    gt_df = cxr_combo_df[(val-20 <= cxr_combo_df['anchor_age']) & (cxr_combo_df['anchor_age'] < val)]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {} (N={})'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values), len(gt_df)))

for val in cxr_combo_df['ethnicity'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['ethnicity'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))

for val in cxr_combo_df['insurance'].unique():
    gt_df = cxr_combo_df[cxr_combo_df['insurance'] == val]
    gt_df = gt_df[['Path', 'study_id', 'Pneumothorax', 'Pneumothorax_pred']].groupby('study_id').mean()[['Pneumothorax', 'Pneumothorax_pred']]
    if len(gt_df['Pneumothorax'].values) == 0 or len(np.unique(gt_df['Pneumothorax'].values)) == 1:
        continue
    print('{}: {}'.format(val, bootstrap_auc(gt_df['Pneumothorax'].values, gt_df['Pneumothorax_pred'].values)))



F: 0.858 ± 0.022 (N=10501)
M: 0.877 ± 0.015 (N=11387)
20: 0.984 ± 0.026 (N=79)
40: 0.895 ± 0.027 (N=2398)
60: 0.884 ± 0.020 (N=7103)
80: 0.860 ± 0.021 (N=8949)
100: 0.825 ± 0.045 (N=3359)
BLACK/AFRICAN AMERICAN: 0.828 ± 0.046
WHITE: 0.871 ± 0.015
HISPANIC/LATINO: 0.868 ± 0.069
OTHER: 0.857 ± 0.039
ASIAN: 0.891 ± 0.040
AMERICAN INDIAN/ALASKA NATIVE: 0.936 ± 0.086
Other: 0.871 ± 0.013
Medicaid: 0.883 ± 0.033
